# 🧼 Rensing og klargjøring av data

I denne notebooken klargjør vi de innhentede datasettene fra NILU og Frost for videre analyse og visualisering.

Vi utfører:
- Strukturering av data
- Fjerning av feil og outliers
- Behandling av manglende verdier

All renselogikk ligger i egne funksjoner og moduler


In [ ]:
#Kjør kode for å rense data

## 🧽 Om rensing og datakvalitet

Etter at vi har hentet inn data fra Frost (MET) og NILU, er neste steg å gjøre dataene klare til analyse. Dette handler både om å forbedre kvaliteten på dataene – og om å vise at vi forstår hvordan ulike datakilder stiller ulike krav til rensing.

Vi valgte to datasett med kontrasterende egenskaper:

- **Frost**: Strukturert, stabilt og nesten «analyseklart»
- **NILU**: Ujevnt, med mange manglende verdier og måleutfordringer

Denne kontrasten gir oss mulighet til å vise både enkel validering og mer avansert rensing og imputasjon.

---

### 🌫️ NILU – utfordrende, men verdifullt

Datasettet fra NILU inneholder daglige målinger av luftkvalitet i Trondheim (PM10, PM2.5, NO₂), men er preget av:

- Mange manglende verdier
- Ekstreme målinger og støy
- Ujevn dekning over tid og mellom komponenter

Samtidig er dette realistiske utfordringer ved miljødata, og gir oss muligheten til å vise god databehandling i praksis.

#### 🔧 Renseprosessen

1. **Konvertering av datoer**  
`dateTime` ble konvertert til `datetime`-format for å gjøre videre tidsseriebehandling mulig (f.eks. resampling og glatting).

2. **Pivotering av datastruktur**  
Målingene ble transformert slik at én rad tilsvarer én dag, og hver luftkomponent fikk sin egen kolonne. Dette forenkler all videre analyse.

3. **Fjerning av kolonne**  
`Benzo(a)pyrene in PM10` ble fjernet fordi den hadde nesten bare manglende verdier, og ikke er relevant for vår problemstilling.

4. **Fjerning av outliers**  
Målinger som lå mer enn 4 standardavvik fra gjennomsnittet ble fjernet. Disse kan være feil eller kortvarige, ekstreme hendelser som ikke representerer typiske forhold.

5. **Reindeksering av datoer**  
Alle datoer mellom første og siste registrering ble inkludert, også de uten målinger. Dette gjør eventuelle datamangler synlige og muliggjør presis imputasjon.

6. **KNN-imputasjon (k = 100)**  
Manglende verdier ble fylt inn med KNN-imputasjon, som bruker lignende dager til å estimere manglende verdier. Mer om KNN-impulasjon og hvorfor vi har valgt dette kan du lese [her](KNN_imputation.ipynb)


7. **Merking av estimerte verdier**  
Vi opprettet egne `generated_*`-kolonner for å vise hvilke verdier som er estimert. Dette gjør datagrunnlaget transparent og analyserbart.

8. **Glidende gjennomsnitt**  
Vi brukte et 3-dagers glidende gjennomsnitt for å jevne ut tilfeldige variasjoner og fremheve trender.  
– *Hvorfor?* Det gir mer lesbare grafer og et tydeligere bilde av utvikling, men uten å skjule kortsiktige endringer.

9. **Negative verdier satt til 0**  
Luftforurensningsverdier kan ikke være negative. Disse feilene ble rettet for å bevare datasettets troverdighet.

10. **Lagring som JSON**  
Renset data ble lagret som `.json`.  
– Lett å lese og bruke videre i både analyse og visualisering.

---

### 🌦️ Frost – strukturert, men fortsatt valideringsbehov

Frost-dataene er langt mer komplette og standardiserte, men ble likevel validert og bearbeidet for å sikre pålitelighet – og for å kunne kombineres med NILU.

#### 🔧 Renseprosessen

1. **Konvertering av datoformat**  
`referenceTime` ble formatert til datoobjekt for å kunne brukes i tidsserier.

2. **Utvalg av relevante variabler**  
Vi fokuserte på:
  - Temperatur (gjennomsnitt per dag)
  - Nedbør (total per dag)
  - Vindhastighet (gjennomsnitt per dag)

  Disse har dokumentert sammenheng med luftkvalitet. Andre mer usikre eller tekniske variabler ble utelatt.

3. **Pivotering og navneendring**  
Datasettet ble omstrukturert til én rad per dag med mer intuitive kolonnenavn.

4. **Verdikontroll med `ValueRangeValidator`**  
Vi definerte gyldige verdier for Trondheim:
  - Temp: -30 til 40 °C  
  - Nedbør: 0 til 250 mm  
  - Vind: 0 til 60 m/s  

  Verdier utenfor ble fjernet – de er trolig feilregistreringer.

5. **Kontroll for dato-hull**  
Med `DateContinuityValidator` sjekket vi at det ikke manglet perioder i tidsserien. Små hull ble registrert, men ikke imputert – siden Frost generelt har god datadekning.

6. **Lagring som SQLite**  
Værdataene ble lagret i en SQLite-database (`frost.db`) – ideelt for videre analyse med SQL og for eventuell samkjøring med større datasett.

---

### 🧠 Hvorfor dette er en god tilnærming

Vi har valgt rensestrategier basert på datakildenes egenskaper:

- **NILU** krevde:
  - Komplett rekonstruering av datastrukturen
  - Avansert imputasjon og glatting
  - Transparens rundt hvilke verdier som er estimert

- **Frost** krevde:
  - Kontroll av gyldige verdier og datoer
  - Lett justering og tilrettelegging for videre analyse

Dette viser:
- Forståelse av hva som påvirker datakvalitet
- Evne til å tilpasse metoder etter utfordring
- Fokus på transparens og sporbarhet

Resultatet er et datasett som både er **ryddet og dokumentert**, og klart for videre analyse, visualisering og modellering.

---

## 📈 Neste steg: Analyse og visualisering

Nå som begge datasett er renset og klargjort, er vi klare for å utforske innholdet mer inngående. I neste notebook skal vi:

- Beregne statistiske mål som gjennomsnitt, median og standardavvik
- Undersøke sammenhenger og trender i luftkvalitet og værdata over tid
- Lage visuelle fremstillinger som gjør datamønstre lettere å forstå


### [**Videre til analyse og visualisering**](02_data_analysis_and_visualisation.ipynb)
##### [**Til samlesiden**](../docs/samleside.md)

